In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline


C:\Users\ALEX\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner", do_lower_case=False)
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = "Настоящая документация подготовлена в соответствии с ГК РФ, Законом № 223-ФЗ, Постановлением Правительства РФ № 925."

ner_results = nlp(example)
print(ner_results)


C:\Users\ALEX\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\pipelines\token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'ORG', 'score': 0.85501224, 'word': 'ГК РФ', 'start': 53, 'end': 58}, {'entity_group': 'ORG', 'score': 0.731745, 'word': 'Правительства РФ', 'start': 93, 'end': 109}]


In [16]:
batch_sentences = [

    "Документация о закупке (документация) – настоящая документация, содержащая установленные Законом № 223-ФЗ и Положением о закупках сведения о запросе предложений.",

    "Извещение о закупке – документ, формируемый посредством функционала ЕИС и содержащий установленные Законом № 223-ФЗ и Положением о закупках сведения о запросе предложений.",

    "Регламент работы ЭТП – документы оператора ЭТП, регламентирующие порядок проведения закупок на ЭТП в соответствии с Законом № 223-ФЗ, определяющие деятельность оператора ЭТП по обеспечению проведения закупок в соответствии с Законом № 223-ФЗ. ",

]

In [17]:
encoded_input = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)

print(encoded_input)

{'input_ids': tensor([[   101,  14418,  11191,  57772,  12942,    555,  10234,  86613,  11557,
            113,  85065,  38878,    114,    100,  32001,  10752,  10385,  26325,
          85065,  38878,    117,  10956,  22605,  17254,  26325,  68855,  11194,
          94493,   1779,  24415,    118,  95689,    549,  45784,  63935,  28788,
            555,  10234,  86613,  20265,  71088,    555,  10234,  99842,  10205,
          23807,  99187,  11550,    119,    102,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0],
        [   101,  19597,  14149,  26692,    555,  10234,  86613,  11557,    100,
          85065,    117,  40391,  14360,  69167,  87953,    561,  16657,  10510,
          86228,  52486,    514,  27735,  14791,    549,  10956,  22605,  17254,
          27312,  68855,  11194,  94493,   1779,  24415,    118,  95689,    549,
          45784,  639

In [24]:
tokenized_sentences = encoded_input["input_ids"]

converted_sentences = [tokenizer.convert_ids_to_tokens(ids) for ids in tokenized_sentences]

for tokens in converted_sentences:
    print(tokens)

['[CLS]', 'До', '##ку', '##мента', '##ция', 'о', 'за', '##куп', '##ке', '(', 'документ', '##ация', ')', '[UNK]', 'нас', '##то', '##я', '##щая', 'документ', '##ация', ',', 'со', '##дер', '##жа', '##щая', 'установлен', '##ные', 'Законом', '№', '223', '-', 'ФЗ', 'и', 'Пол', '##ожен', '##ием', 'о', 'за', '##куп', '##ках', 'сведения', 'о', 'за', '##прос', '##е', 'пред', '##ложен', '##ий', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', 'Из', '##ве', '##щение', 'о', 'за', '##куп', '##ке', '[UNK]', 'документ', ',', 'форми', '##ру', '##емый', 'посредством', 'ф', '##ун', '##к', '##цио', '##нала', 'Е', '##И', '##С', 'и', 'со', '##дер', '##жа', '##щий', 'установлен', '##ные', 'Законом', '№', '223', '-', 'ФЗ', 'и', 'Пол', '##ожен', '##ием', 'о', 'за', '##куп', '##ках', 'сведения', 'о', 'за', '##прос', '##е', 'пред', '##ложен', '##ий', 

In [29]:
# Initialize an empty list to store reconstructed full sentences
reconstructed_sentences = []

# Iterate through the tokenized sentences
for tokens in tokenized_sentences:
    current_sentence = []  # Create an empty list for the current sentence
    for token_id in tokens:
        token = tokenizer.convert_ids_to_tokens([token_id.item()])[0]
        if token.startswith("##"):  # Check if it's a subword token
            current_sentence[-1] += token[2:]  # Append to the last token in the current sentence
        else:
            current_sentence.append(token)  # Start a new token for the current sentence
    reconstructed_sentences.append(current_sentence)  # Add the current sentence to the list

# Print the reconstructed sentences
for sentence in reconstructed_sentences:
    print(sentence)


['[CLS]', 'Документация', 'о', 'закупке', '(', 'документация', ')', '[UNK]', 'настоящая', 'документация', ',', 'содержащая', 'установленные', 'Законом', '№', '223', '-', 'ФЗ', 'и', 'Положением', 'о', 'закупках', 'сведения', 'о', 'запросе', 'предложений', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', 'Извещение', 'о', 'закупке', '[UNK]', 'документ', ',', 'формируемый', 'посредством', 'функционала', 'ЕИС', 'и', 'содержащий', 'установленные', 'Законом', '№', '223', '-', 'ФЗ', 'и', 'Положением', 'о', 'закупках', 'сведения', 'о', 'запросе', 'предложений', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', 'Регламент', 'работы', 'ЭТП', '[UNK]', 'документы', 'оператора', 'ЭТП', ',', 'регламентиру